In [ ]:
import pygame
from mpmath import mp
import numpy as np
from scipy.stats import linregress
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
pygame.display.set_caption("El Examen")

info = pygame.display.Info()
width, height = info.current_w, info.current_h

# Colors
white = (255, 255, 255)
gray = (211, 211, 211)
black = (0, 0, 0)

#Font
font = pygame.font.Font(None,32)
font_2 = pygame.font.Font(None,25)

#Buttons
button_width, button_height = int(width/7), 50

button_color = black
button_math = pygame.Rect(int(width/7), 150, button_width, button_height)
button_chem = pygame.Rect(3*int(width/7), 150, button_width, button_height)
button_eng = pygame.Rect(5*int(width/7), 150, button_width, button_height)


dropdown = pygame.Rect(-10,-20, width/4, height/20) 
home_button = pygame.Rect(50,-30,30,30)
close_button = pygame.Rect(10,-30,30,30)

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second
running = True

def dropdown_dynamics(root, dropdown, home_button, close_button):
    mouse_x, mouse_y = pygame.mouse.get_pos()

    hover = dropdown.collidepoint(mouse_x, mouse_y)
    if hover and dropdown.height < 80 and home_button.y < 10:
        dropdown.height += 5
        home_button.y += 5
        close_button.y += 5 
    elif not hover and dropdown.height > 40 and home_button.y >-30:
        dropdown.height -= 5
        home_button.y -= 5
        close_button.y -= 5

    dropdown = pygame.Rect(-10,-20, 200, dropdown.height) 
    pygame.draw.rect(root, gray, dropdown, 0, 20)

    home_button = pygame.Rect(50, home_button.y,30,30)
    pygame.draw.rect(root, black, home_button,0, 15)
    home_text = font.render("[]", True, white)
    home_text_1 = home_text.get_rect(center=home_button.center)
    root.blit(home_text, home_text_1)

    close_button = pygame.Rect(10, close_button.y,30,30)
    pygame.draw.rect(root, black, close_button,0, 15)
    close_text = font.render("X", True, white)
    close_text_1 = close_text.get_rect(center=close_button.center)
    root.blit(close_text, close_text_1)


def button_dynamics(button, name, subname_1, subname_2, subname_3):
        hover = button.collidepoint(mouse_x, mouse_y)

        if hover and button.height < 350:
            button.height += 10
        elif not hover and button.height > 50:
            button.height -= 15
        
        button = pygame.Rect(button.x, button.y, button.width, button.height)
        pygame.draw.rect(screen, black, button,0, 30)

        text = font.render(name, True, white)
        text_button = text.get_rect(center=(button.centerx, button.y+25))
        screen.blit(text, text_button)

        subbutton_1 = subbutton_2 = subbutton_3 = None

        if button.height > 110:
            subbutton_1 = pygame.Rect(button.x+10,button.y+60,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_1,0, 30)
            text_1 = font_2.render(subname_1, True, black)
            text_subbutton_1 = text_1.get_rect(center=subbutton_1.center)
            screen.blit(text_1, text_subbutton_1)

        if button.height > 210:
            subbutton_2 = pygame.Rect(button.x+10,button.y+160,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_2,0, 30)
            text_2 = font_2.render(subname_2, True, black)
            text_subbutton_2 = text_2.get_rect(center=subbutton_2.center)
            screen.blit(text_2, text_subbutton_2)

        if button.height > 310:
            subbutton_3 = pygame.Rect(button.x+10,button.y+260,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_3,0, 30)
            text_3 = font_2.render(subname_3, True, black)
            text_subbutton_3 = text_3.get_rect(center=subbutton_3.center)
            screen.blit(text_3, text_subbutton_3)
        
        return subbutton_1, subbutton_2, subbutton_3

def generate_points_with_correlation(R, n_points=100):
    # Generate random x values
    x = np.random.randn(n_points)
    
    # Generate random noise
    noise = np.random.randn(n_points)
    
    # Generate y values that will have the desired correlation with x
    y = R * x + np.sqrt(1 - R**2) * noise
    
    return x, y

def Math_Game_Uno():
    global running
    math_game_uno = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
    pygame.display.set_caption('Math uno')
    clock = pygame.time.Clock()
    fps = 60  # Set the desired frames per second

    # Define colors
    white = (255, 255, 255)
    black = (0, 0, 0)
    blue = (0, 0, 255)
    gray = (200, 200, 200)

    # Define font
    font = pygame.font.Font(None, 36)

    # Define input box
    input_box = pygame.Rect((width-500)-140, height-600, 140, 32)
    color_inactive = pygame.Color('gray64')
    color_active = pygame.Color('gray17')
    color = color_inactive
    active = False
    text = '0.'

    R = random.randint(0, 100)/100
    # Generate the points
    x, y = generate_points_with_correlation(R)

    # Define guess button
    guess_button = pygame.Rect(width-500, height-600, 100, 32)
    guessing = True

    # Define result text
    true_R_box = pygame.Rect(width-700, height-550, 100, 32)

    # Define next button
    next_button = pygame.Rect(width-700, height-600, 100, 32)

    while True:
        math_game_uno.fill(white)
        mouse_x, mouse_y = pygame.mouse.get_pos()

        # Draw square outline of the graph around the middle of the window
        window_width, window_height = math_game_uno.get_size()
        graph_size = min(window_width, window_height) // 2
        graph_x = (window_width - graph_size) // 2 - 300
        graph_y = (window_height - graph_size) // 2
        pygame.draw.rect(math_game_uno, 'gray64', (graph_x, graph_y, graph_size, graph_size), 2)
        pygame.draw.line(math_game_uno, 'gray64', (graph_x, graph_y + graph_size // 2), (graph_x + graph_size, graph_y + graph_size // 2), 2)
        pygame.draw.line(math_game_uno, 'gray64', (graph_x + graph_size // 2, graph_y), (graph_x + graph_size // 2, graph_y + graph_size), 2)
        

        # Plot newly generated points
        plot_points_with_pygame(x, y, math_game_uno, graph_x, graph_y, graph_size)

        

        if guessing:
            # Draw input box
            txt_surface = font.render(text, True, color)
            input_box.w = max(200, txt_surface.get_width() + 10)
            math_game_uno.blit(txt_surface, (input_box.x + 5, input_box.y + 5))
            pygame.draw.rect(math_game_uno, color, input_box, 3)

            # Draw guess button
            pygame.draw.rect(math_game_uno, blue, guess_button)
            guess_text = font.render("Guess", True, white)
            guess_text_rect = guess_text.get_rect(center=guess_button.center)
            math_game_uno.blit(guess_text, guess_text_rect)
        else:
            # next button
            pygame.draw.rect(math_game_uno, blue, next_button)
            next_text = font.render("Next", True, white)
            next_text_rect = next_text.get_rect(center=next_button.center)
            math_game_uno.blit(next_text, next_text_rect)

            # true R text button
            pygame.draw.rect(math_game_uno, gray, true_R_box)
            true_R_text = font.render("True R:  0.x", True, black)
            true_R_box_rect = true_R_text.get_rect(left=true_R_box.center)
            math_game_uno.blit(true_R_text, true_R_box_rect)

        dropdown_dynamics(math_game_uno, dropdown, home_button, close_button)

        for event in pygame.event.get():
            # Makes the dropdown_dynamics function work
            if home_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                return
            if close_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                running = False
                return
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                return
            
            # Primary code for mouse interactions
            if event.type == pygame.MOUSEBUTTONDOWN:
                if input_box.collidepoint(event.pos):
                    active = not active
                else:
                    active = False
                # INterraction with guess box
                if guess_button.collidepoint(event.pos):
                    if text[2:].isdigit(): 
                        guessing = False
                        #x, y = generate_points_with_correlation(float(text))
                        text = '0.'
                        active = False
                    else:
                        text = '0.'
                        active = False 
                # Interaction with next button
                if next_button.collidepoint(event.pos):
                    R = random.randint(0, 100)/100
                    x, y = generate_points_with_correlation(R)
                    guessing = True
                    text = '0.'
                    active = False

            color = color_active if active else color_inactive

            if event.type == pygame.KEYDOWN:
                if active:
                    if event.key == pygame.K_RETURN:
                        if text[2:].isdigit():
                            guessing = False
                            #x, y = generate_points_with_correlation(float(text))
                            text = '0.'
                            active = False
                        else:
                            text = '0.'
                            active = False                            
                    elif event.key == pygame.K_BACKSPACE:
                        text = text[:-1]
                    else:
                        text += event.unicode

        pygame.display.flip()
        # Cap the frame rate to 60 fps
        clock.tick(fps)

def plot_points_with_pygame(x, y, screen, graph_x, graph_y, graph_size):
    # Define colors
    black = (0, 0, 0)
    red = (255, 0, 0)
    
    # Scale the points to fit the screen
    x_scaled = (x - x.min()) / (x.max() - x.min()) * graph_size + graph_x
    y_scaled = height - (y - y.min()) / (y.max() - y.min()) * graph_size - graph_y
    
    # Calculate the regression line
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    x_min, x_max = x.min(), x.max()
    y_min = slope * x_min + intercept
    y_max = slope * x_max + intercept
    
    x_min_scaled = (x_min - x.min()) / (x.max() - x.min()) * graph_size + graph_x
    x_max_scaled = (x_max - x.min()) / (x.max() - x.min()) * graph_size + graph_x
    y_min_scaled = height - (y_min - y.min()) / (y.max() - y.min()) * graph_size - graph_y
    y_max_scaled = height - (y_max - y.min()) / (y.max() - y.min()) * graph_size - graph_y
       
    # Draw the points
    for i in range(len(x_scaled)):
        pygame.draw.circle(screen, black, (int(x_scaled[i]), int(y_scaled[i])), 3)
        
    # Draw the regression line
    pygame.draw.line(screen, red, (int(x_min_scaled), int(y_min_scaled)), (int(x_max_scaled), int(y_max_scaled)), 2)

def Math_Game_Dos():
    global running
    math_game_dos = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
    pygame.display.set_caption('Constant Runner')
    clock = pygame.time.Clock()
    fps = 60  # Set the desired frames per second

    text = font_2.render("3.", True, white)
    n=3
    
    numberkeys ={
            pygame.K_1: "1",
            pygame.K_2: "2",
            pygame.K_3: "3",
            pygame.K_4: "4",
            pygame.K_5: "5",
            pygame.K_6: "6",
            pygame.K_7: "7",
            pygame.K_8: "8",
            pygame.K_9: "9"
        }

    while True:
        math_game_dos.fill(white)
        mouse_x, mouse_y = pygame.mouse.get_pos()
        dropdown_dynamics(math_game_dos, dropdown, home_button, close_button)
        
        button_checker = pygame.Rect(width/2-100, height/2-100, 200, 200)
        pygame.draw.rect(math_game_dos, black, button_checker,0, 30)

        mp.dps = n + 2
        pi = str(mp.pi)
        pi_n = pi[n-1:n]


        for event in pygame.event.get():
            # Makes the dropdown_dynamics function work
            if home_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                return
            if close_button.collidepoint(mouse_x,mouse_y) and event.type == pygame.MOUSEBUTTONDOWN:
                running = False
                return
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                return
            # From here you can fuck around ~~~~~~~~~~~
            if event.type == pygame.KEYDOWN:
                if event.key in numberkeys:
                    if numberkeys[event.key] == pi_n:
                        text = font_2.render(str(pi[:n]), True, white)
                        n += 1
        
        text_math_2 = text.get_rect(center=button_checker.center)
        math_game_dos.blit(text, text_math_2)
        
        pygame.display.flip()
        pygame.display.update()
        
        # Cap the frame rate to 60 fps
        clock.tick(fps)

def Math_Game_Tres():
    button_3 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_3,0, 30)
    text_math_3 = font_2.render("Math Game 3", True, white)
    text_math_1_3 = text_math_3.get_rect(center=button_3.center)
    screen.blit(text_math_3, text_math_1_3)

def Eng_Game_Uno():  
    button_4 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_4,0, 30)
    text_eng_1 = font_2.render("English Game 1", True, white)
    text_eng_1_1 = text_eng_1.get_rect(center=button_4.center)
    screen.blit(text_eng_1, text_eng_1_1)

def Eng_Game_Dos(): 
    button_5 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_5,0, 30)
    text_eng_2 = font_2.render("English Game 2", True, white)
    text_eng_1_2 = text_eng_2.get_rect(center=button_5.center)
    screen.blit(text_eng_2, text_eng_1_2)

def Eng_Game_Tres(): 
    button_6 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_6,0, 30)
    text_eng_3 = font_2.render("English Game 3", True, white)
    text_eng_1_3 = text_eng_3.get_rect(center=button_6.center)
    screen.blit(text_eng_3, text_eng_1_3)

def Chem_Game_Uno(): 
    button_7 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_7,0, 30)
    text_chem_1 = font_2.render("Chemistry Game 1", True, white)
    text_chem_1_1 = text_chem_1.get_rect(center=button_7.center)
    screen.blit(text_chem_1, text_chem_1_1) 

def Chem_Game_Dos(): 
    button_8 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_8,0, 30)
    text_chem_2 = font_2.render("Chemistry Game 2", True, white)
    text_chem_1_2 = text_chem_2.get_rect(center=button_8.center)
    screen.blit(text_chem_2, text_chem_1_2) 

def Chem_Game_Tres(): 
    button_9 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_9,0, 30)
    text_chem_3 = font_2.render("Chemistry Game 3", True, white)
    text_chem_1_3 = text_chem_3.get_rect(center=button_9.center)
    screen.blit(text_chem_3, text_chem_1_3) 


while running:
    screen.fill(white)
    mouse_x, mouse_y = pygame.mouse.get_pos()

    dropdown_dynamics(screen, dropdown, home_button, close_button)

    subbutton_math_1, subbutton_math_2, subbutton_math_3 = button_dynamics(button_math, 'Math','Game 1','Game 2','Game 3')
    subbutton_chem_1, subbutton_chem_2, subbutton_chem_3 = button_dynamics(button_chem, 'Chemistry','Game 1','Game 2','Game 3')
    subbutton_eng_1, subbutton_eng_2, subbutton_eng_3 = button_dynamics(button_eng,'English','Game 1','Game 2','Game 3')
    
    for event in pygame.event.get():
        if event.type == pygame.MOUSEBUTTONDOWN:
                if close_button.collidepoint(mouse_x,mouse_y):
                    running = False
                if subbutton_math_1 and subbutton_math_1.collidepoint(mouse_x, mouse_y):
                    Math_Game_Uno()
                if subbutton_math_2 and subbutton_math_2.collidepoint(mouse_x, mouse_y):
                    Math_Game_Dos()
                if subbutton_math_3 and subbutton_math_3.collidepoint(mouse_x, mouse_y):
                    Math_Game_Tres()
                if subbutton_chem_1 and subbutton_chem_1.collidepoint(mouse_x, mouse_y):
                    Chem_Game_Uno()
                if subbutton_chem_2 and subbutton_chem_2.collidepoint(mouse_x, mouse_y):
                    Chem_Game_Dos()
                if subbutton_chem_3 and subbutton_chem_3.collidepoint(mouse_x, mouse_y):
                    Chem_Game_Tres()
                if subbutton_eng_1 and subbutton_eng_1.collidepoint(mouse_x, mouse_y):
                    Eng_Game_Uno()
                if subbutton_eng_2 and subbutton_eng_2.collidepoint(mouse_x, mouse_y):
                    Eng_Game_Dos()
                if subbutton_eng_3 and subbutton_eng_3.collidepoint(mouse_x, mouse_y):
                    Eng_Game_Tres()
                
# Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


TypeError: invalid rect assignment

: 

In [1]:
pip install matplotlib

     ---------------------------------------- 0.0/104.0 kB ? eta -:--:--
     ----------- --------------------------- 30.7/104.0 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 104.0/104.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 10.0 MB/s eta 0:00:01
   ------ --------------------------------- 1.2/8.1 MB 13.1 MB/s eta 0:00:01
   --------- ------------------------------ 2.0/8.1 MB 14.0 MB/s eta 0:00:01
   ------------- -------------------------- 2.6/8.1 MB 14.0 MB/s eta 0:00:01
   ----------------- ---------------------- 3.5/8.1 MB 14.8 MB/s eta 0:00:01
   -------------------- ------------------- 4.1/8.1 MB 14.7 MB/s eta 0:00:01
   ------------------------ --------------- 5.0/8.1 MB 15.1 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.1 MB 15.4 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 15.5 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\kjaer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
